In [1]:
#% pylab inline
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LSTM, Bidirectional
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History

Using TensorFlow backend.
C:\Users\teres\Anaconda3\envs\py3-TF1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\teres\Anaconda3\envs\py3-TF1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\teres\Anaconda3\envs\py3-TF1.14\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\teres\Anaconda3\envs\py

In [ ]:
#originally def get_notes() code in GAN
notes = []
midi = converter.parse("anger_simple/Phantom_of_the_opera_pifa.mid")

#print("Parsing %s" % file)

notes_to_parse = None

try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse()
    notes_to_parse.show('text')
    notes_to_parse.show('musicxml')
except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

print("len notes:", len(notes))
print('len set notes: ', len(set(notes)))
   # return notes

{0.0} <music21.instrument.Piano 'Piano'>
{0.0} <music21.tempo.MetronomeMark animato Quarter=120.0>
{0.0} <music21.key.Key of C major>
{0.0} <music21.meter.TimeSignature 4/4>
{0.0} <music21.note.Rest rest>
{4.0} <music21.chord.Chord E-4 G4 C5>
{9.6667} <music21.note.Rest rest>
{10.0} <music21.chord.Chord E-4 G4 C5>
{10.5} <music21.chord.Chord D4 F#4 B4>
{11.0} <music21.chord.Chord C#4 F4 B-4>
{11.5} <music21.chord.Chord C#4 E4 A4>
{12.0} <music21.chord.Chord C4 E-4 G#4>
{15.75} <music21.note.Rest rest>
{16.0} <music21.chord.Chord B-3 C#4 F#4>
{16.5} <music21.chord.Chord C4 E-4 G#4>
{18.5} <music21.note.Rest rest>
{20.0} <music21.chord.Chord E-4 G4 C5>
{25.6667} <music21.note.Rest rest>
{26.0} <music21.chord.Chord E-4 G4 C5>
{26.5} <music21.chord.Chord D4 F#4 B4>
{27.0} <music21.chord.Chord C#4 F4 B-4>
{27.5} <music21.chord.Chord C#4 E4 A4>
{28.0} <music21.chord.Chord C4 E-4 G#4>
{31.75} <music21.note.Rest rest>
{32.0} <music21.chord.Chord B-3 C#4 F#4>
{32.5} <music21.chord.Chord C4 E-4 

In [4]:
#originally def get_notes() code in GAN
notes = []
for file in glob.glob("love_obpi/*.mid"):
    midi = converter.parse(file)

    print("Parsing %s" % file)

    notes_to_parse = None

    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
            
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

print("len notes:", len(notes))
print('len set notes: ', len(set(notes)))
   # return notes

Parsing love_obpi\Advance_To_The_Rear_obpi.mid
Parsing love_obpi\All_Out_Of_Love_obpi.mid
Parsing love_obpi\As_Time_Goes_By_obpi.mid
Parsing love_obpi\Dr_Zhivago_obpi.mid
Parsing love_obpi\Legends_Of_The_Fall_obpi.mid
Parsing love_obpi\Out_Of_Africa_obpi.mid
len notes: 1061
len set notes:  86


In [10]:
#originally def prepare_sequences(notes, n_vocab):

#set number of vocabs to the number of different notes found in the corpus
n_vocab = len(set(notes))

#""" Prepare the sequences used by the Neural Network """
sequence_length = 100

# Get all pitch names
pitchnames = sorted(set(item for item in notes))

# Create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
print("n_patterns: ", n_patterns)

# Reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

print("network_input after reshape:", network_input)
print("# columns: ", len(network_input[0]))
print('# rows: ', len(network_input))
# normalize input between 0 and 1 
network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)
print('network_output', network_output)
print("# columns: ", len(network_output[0]))
print('# rows: ', len(network_output))

   # return (network_input, network_output)
X_train, y_train = network_input, network_output

n_patterns:  961
network_input after reshape: [[[84]
  [69]
  [61]
  ...
  [69]
  [77]
  [61]]

 [[69]
  [61]
  [58]
  ...
  [77]
  [61]
  [58]]

 [[61]
  [58]
  [77]
  ...
  [61]
  [58]
  [51]]

 ...

 [[44]
  [83]
  [71]
  ...
  [57]
  [44]
  [83]]

 [[83]
  [71]
  [60]
  ...
  [44]
  [83]
  [44]]

 [[71]
  [60]
  [68]
  ...
  [83]
  [44]
  [58]]]
# columns:  100
# rows:  961
network_output [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
# columns:  86
# rows:  961


In [12]:
#Version 1: lstm-gan
#originally def create_network(network_input, n_vocab):
  #  """ create the structure of the neural network """
model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

   #return model
history = History()

In [25]:
#Version 2: lstm
#originally def create_network(network_input, n_vocab):
#""" create the structure of the neural network """
model = Sequential()
model.add(LSTM(
    512,#512
    input_shape=(network_input.shape[1], network_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))
model.add(LSTM(256, return_sequences=True, recurrent_dropout=0.3,))#512
model.add(LSTM(512))#512
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128)) #256
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
#zum Gewichte laden einkommentieren
#model.load_weights("weights-improvement-03-4.6675-bigger.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
print("Created model and loaded weights from file")
#return model

history = History()

Created model and loaded weights from file


In [13]:
#Version 1
#new fit network method

n_epochs = 2
model.summary()
model.fit(network_input, network_output, callbacks=[history], epochs=n_epochs, batch_size=64)
model.save('LSTMmodel.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 1024)         4198400   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              6295552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [28]:
#Version 2 mit Checkpoints und Gewichte speichern
#originally def train(model, network_input, network_output):
 #   """ train the neural network """
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min',
    period=2
)
callbacks_list = [checkpoint]

model.summary()
model.fit(network_input, network_output, epochs=4, batch_size=128, callbacks=callbacks_list)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 100, 512)          1052672   
_________________________________________________________________
lstm_14 (LSTM)               (None, 100, 256)          787456    
_________________________________________________________________
lstm_15 (LSTM)               (None, 512)               1574912   
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               65664     
_________________________________________________________________
activation_3 (Activation)    (None, 128)              

In [16]:
#originally def generate_notes(model, notes, network_input, n_vocab):
   # """ Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
pitchnames = sorted(set(item for item in notes))

start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

#pick 1 sequences from network_input at position=start
pattern = network_input[start]
print("pattern:",pattern)
print("pattern len:", len(pattern))
prediction_output = []

# generate 500 notes
for note_index in range(500):
    #reshape for lstm-layer compability
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    print("prediction_input after reshaping", prediction_input)
    #normalize input between 0 and 1
    prediction_input = prediction_input / float(n_vocab)
    print("prediction_input after normalization", prediction_input)

    prediction = model.predict(prediction_input, verbose=0)

    index = np.argmax(prediction)
    print("index:", index)
    result = int_to_note[index]
    print("result:", result)
    prediction_output.append(result)

    pattern = np.append(pattern,index)
    pattern = pattern[1:len(pattern)]

#return prediction_output

pattern: [[0.54651163]
 [0.20930233]
 [0.6627907 ]
 [0.74418605]
 [0.69767442]
 [0.87209302]
 [0.55813953]
 [0.55813953]
 [0.6627907 ]
 [0.55813953]
 [0.87209302]
 [0.86046512]
 [0.74418605]
 [0.95348837]
 [0.55813953]
 [0.55813953]
 [0.74418605]
 [0.95348837]
 [0.02325581]
 [0.6627907 ]
 [0.88372093]
 [0.87209302]
 [0.6627907 ]
 [0.88372093]
 [0.55813953]
 [0.75581395]
 [0.95348837]
 [0.69767442]
 [0.95348837]
 [0.75581395]
 [0.95348837]
 [0.69767442]
 [0.73255814]
 [0.74418605]
 [0.38372093]
 [0.74418605]
 [0.55813953]
 [0.55813953]
 [0.95348837]
 [0.55813953]
 [0.87209302]
 [0.55813953]
 [0.6627907 ]
 [0.20930233]
 [0.54651163]
 [0.69767442]
 [0.69767442]
 [0.55813953]
 [0.87209302]
 [0.45348837]
 [0.20930233]
 [0.95348837]
 [0.54651163]
 [0.55813953]
 [0.6627907 ]
 [0.69767442]
 [0.55813953]
 [0.53488372]
 [0.55813953]
 [0.87209302]
 [0.69767442]
 [0.6627907 ]
 [0.87209302]
 [0.55813953]
 [0.74418605]
 [0.55813953]
 [0.55813953]
 [0.54651163]
 [0.74418605]
 [0.95348837]
 [0.9534883

index: 84
result: G5
prediction_input after reshaping [[[6.62790698e-01]
  [7.44186047e-01]
  [6.97674419e-01]
  [8.72093023e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [6.62790698e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [8.60465116e-01]
  [7.44186047e-01]
  [9.53488372e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [7.44186047e-01]
  [9.53488372e-01]
  [2.32558140e-02]
  [6.62790698e-01]
  [8.83720930e-01]
  [8.72093023e-01]
  [6.62790698e-01]
  [8.83720930e-01]
  [5.58139535e-01]
  [7.55813953e-01]
  [9.53488372e-01]
  [6.97674419e-01]
  [9.53488372e-01]
  [7.55813953e-01]
  [9.53488372e-01]
  [6.97674419e-01]
  [7.32558140e-01]
  [7.44186047e-01]
  [3.83720930e-01]
  [7.44186047e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [9.53488372e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [5.58139535e-01]
  [6.62790698e-01]
  [2.09302326e-01]
  [5.46511628e-01]
  [6.97674419e-01]
  [6.97674419e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [4.53488372e-01]
  [2.09302326e-01]
  [9.53488372e-

index: 75
result: F3
prediction_input after reshaping [[[8.72093023e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [6.62790698e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [8.60465116e-01]
  [7.44186047e-01]
  [9.53488372e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [7.44186047e-01]
  [9.53488372e-01]
  [2.32558140e-02]
  [6.62790698e-01]
  [8.83720930e-01]
  [8.72093023e-01]
  [6.62790698e-01]
  [8.83720930e-01]
  [5.58139535e-01]
  [7.55813953e-01]
  [9.53488372e-01]
  [6.97674419e-01]
  [9.53488372e-01]
  [7.55813953e-01]
  [9.53488372e-01]
  [6.97674419e-01]
  [7.32558140e-01]
  [7.44186047e-01]
  [3.83720930e-01]
  [7.44186047e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [9.53488372e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [5.58139535e-01]
  [6.62790698e-01]
  [2.09302326e-01]
  [5.46511628e-01]
  [6.97674419e-01]
  [6.97674419e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [4.53488372e-01]
  [2.09302326e-01]
  [9.53488372e-01]
  [5.46511628e-01]
  [5.58139535e-01]
  [6.62790698e-

index: 75
result: F3
prediction_input after reshaping [[[6.62790698e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [8.60465116e-01]
  [7.44186047e-01]
  [9.53488372e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [7.44186047e-01]
  [9.53488372e-01]
  [2.32558140e-02]
  [6.62790698e-01]
  [8.83720930e-01]
  [8.72093023e-01]
  [6.62790698e-01]
  [8.83720930e-01]
  [5.58139535e-01]
  [7.55813953e-01]
  [9.53488372e-01]
  [6.97674419e-01]
  [9.53488372e-01]
  [7.55813953e-01]
  [9.53488372e-01]
  [6.97674419e-01]
  [7.32558140e-01]
  [7.44186047e-01]
  [3.83720930e-01]
  [7.44186047e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [9.53488372e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [5.58139535e-01]
  [6.62790698e-01]
  [2.09302326e-01]
  [5.46511628e-01]
  [6.97674419e-01]
  [6.97674419e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [4.53488372e-01]
  [2.09302326e-01]
  [9.53488372e-01]
  [5.46511628e-01]
  [5.58139535e-01]
  [6.62790698e-01]
  [6.97674419e-01]
  [5.58139535e-01]
  [5.34883721e-

index: 48
result: B-3
prediction_input after reshaping [[[8.60465116e-01]
  [7.44186047e-01]
  [9.53488372e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [7.44186047e-01]
  [9.53488372e-01]
  [2.32558140e-02]
  [6.62790698e-01]
  [8.83720930e-01]
  [8.72093023e-01]
  [6.62790698e-01]
  [8.83720930e-01]
  [5.58139535e-01]
  [7.55813953e-01]
  [9.53488372e-01]
  [6.97674419e-01]
  [9.53488372e-01]
  [7.55813953e-01]
  [9.53488372e-01]
  [6.97674419e-01]
  [7.32558140e-01]
  [7.44186047e-01]
  [3.83720930e-01]
  [7.44186047e-01]
  [5.58139535e-01]
  [5.58139535e-01]
  [9.53488372e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [5.58139535e-01]
  [6.62790698e-01]
  [2.09302326e-01]
  [5.46511628e-01]
  [6.97674419e-01]
  [6.97674419e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [4.53488372e-01]
  [2.09302326e-01]
  [9.53488372e-01]
  [5.46511628e-01]
  [5.58139535e-01]
  [6.62790698e-01]
  [6.97674419e-01]
  [5.58139535e-01]
  [5.34883721e-01]
  [5.58139535e-01]
  [8.72093023e-01]
  [6.97674419e

...

index: 48
result: B-3
prediction_input after reshaping [[[48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]]]
prediction_input after normalization [[[0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
 

index: 48
result: B-3
prediction_input after reshaping [[[48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]]]
prediction_input after normalization [[[0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
 

index: 48
result: B-3
prediction_input after reshaping [[[48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]]]
prediction_input after normalization [[[0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
 

index: 48
result: B-3
prediction_input after reshaping [[[48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]]]
prediction_input after normalization [[[0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
 

index: 48
result: B-3
prediction_input after reshaping [[[48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]
  [48.]]]
prediction_input after normalization [[[0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
  [0.55813953]
 

In [18]:
#originally def create_midi(prediction_output, filename):
   # """ convert the output from the prediction to notes and create a midi file
  #      from the notes """
filename = 'love_midi'
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5


    
midi_stream = stream.Stream(output_notes)

midi_stream.show('text')
midi_stream.show('musicxml')

#einkommentieren, um midi file zu speichern
#midi_stream.write('midi', fp='{}.mid'.format(filename))

{0.0} <music21.note.Note G>
{0.5} <music21.note.Note G>
{1.0} <music21.note.Note E->
{1.5} <music21.note.Note C>
{2.0} <music21.note.Note F>
{2.5} <music21.note.Note F>
{3.0} <music21.note.Note F>
{3.5} <music21.note.Note F>
{4.0} <music21.note.Note B->
{4.5} <music21.note.Note B->
{5.0} <music21.note.Note B->
{5.5} <music21.note.Note B->
{6.0} <music21.note.Note B->
{6.5} <music21.note.Note B->
{7.0} <music21.note.Note B->
{7.5} <music21.note.Note B->
{8.0} <music21.note.Note B->
{8.5} <music21.note.Note B->
{9.0} <music21.note.Note B->
{9.5} <music21.note.Note B->
{10.0} <music21.note.Note B->
{10.5} <music21.note.Note B->
{11.0} <music21.note.Note B->
{11.5} <music21.note.Note B->
{12.0} <music21.note.Note B->
{12.5} <music21.note.Note B->
{13.0} <music21.note.Note B->
{13.5} <music21.note.Note B->
{14.0} <music21.note.Note B->
{14.5} <music21.note.Note B->
{15.0} <music21.note.Note B->
{15.5} <music21.note.Note B->
{16.0} <music21.note.Note B->
{16.5} <music21.note.Note B->
{17.0} 

In [29]:
 # Plot the model losses
pd.DataFrame(history.history).plot()

#einkommentieren, um Loss zu speichern
#plt.savefig('LSTM_Loss_per_Epoch.png', transparent=True)
#plt.close()

AttributeError: 'History' object has no attribute 'history'